In [5]:
import os
import cv2
import numpy as np
import random
from PIL import Image, ImageEnhance
from tqdm import tqdm

# ========== CONFIG ==========
output_dir = "synthetic_squares_messy"
img_dir = os.path.join(output_dir, "images")
lbl_dir = os.path.join(output_dir, "labels")
os.makedirs(img_dir, exist_ok=True)
os.makedirs(lbl_dir, exist_ok=True)

img_size = 224
total_images = 1000   # Set as needed
min_squares = 1
max_squares = 3
blur_prob = 0.5
brightness_jitter = 0.4

colors = {
    "blue":   ((255, 30, 30), 0),
    "red":    ((30, 30, 255), 1),
    "orange": ((0, 165, 255), 2),
    "yellow": ((0, 255, 255), 3),
    "purple": ((200, 80, 200), 4),
    "brown":  ((19, 69, 139), 5)
}
class_names = list(colors.keys())

# ========== BACKGROUND GENERATOR ==========
def make_background():
    bg = np.full((img_size, img_size, 3), random.randint(100, 200), dtype=np.uint8)

    # Add colored blobs
    for _ in range(random.randint(5, 20)):
        color = tuple(int(x) for x in np.random.randint(50, 255, 3))
        center = (random.randint(0, img_size), random.randint(0, img_size))
        radius = random.randint(10, 50)
        cv2.circle(bg, center, radius, color, -1)

    # Add lines/patterns
    for _ in range(random.randint(3, 10)):
        pt1 = (random.randint(0, img_size), random.randint(0, img_size))
        pt2 = (random.randint(0, img_size), random.randint(0, img_size))
        color = tuple(int(x) for x in np.random.randint(100, 255, 3))
        thickness = random.choice([1, 2])
        cv2.line(bg, pt1, pt2, color, thickness)

    # Add rectangles as distractions
    for _ in range(random.randint(3, 8)):
        top_left = (random.randint(0, img_size - 30), random.randint(0, img_size - 30))
        w, h = random.randint(10, 50), random.randint(10, 50)
        color = tuple(int(x) for x in np.random.randint(0, 200, 3))
        cv2.rectangle(bg, top_left, (top_left[0] + w, top_left[1] + h), color, -1)

    # Simulate vignette
    Y, X = np.ogrid[:img_size, :img_size]
    center = (img_size / 2, img_size / 2)
    dist_from_center = np.sqrt((X - center[0])**2 + (Y - center[1])**2)
    vignette = 1 - 0.6 * (dist_from_center / dist_from_center.max())
    for i in range(3):
        bg[..., i] = np.clip(bg[..., i] * vignette, 0, 255)

    # Simulate camera sensor noise
    noise = np.random.normal(0, 14, (img_size, img_size, 3))
    bg = np.clip(bg + noise, 0, 255).astype(np.uint8)

    return bg


# ========== IMAGE AUGMENTATION ==========
def apply_brightness_contrast(img):
    img = Image.fromarray(img)
    img = ImageEnhance.Brightness(img).enhance(random.uniform(0.7, 1.3))
    img = ImageEnhance.Contrast(img).enhance(random.uniform(0.7, 1.3))
    return np.array(img)

def apply_motion_blur(image, size=5, angle=0):
    kernel = np.zeros((size, size))
    kernel[int((size - 1) / 2), :] = np.ones(size)
    M = cv2.getRotationMatrix2D((size / 2 - 0.5, size / 2 - 0.5), angle, 1.0)
    kernel = cv2.warpAffine(kernel, M, (size, size))
    kernel /= np.sum(kernel)
    return cv2.filter2D(image, -1, kernel)

def apply_blur(img):
    if random.random() < blur_prob:
        ksize = random.choice([3, 5, 7])
        img = cv2.GaussianBlur(img, (ksize, ksize), 0)
        if random.random() < 0.3:
            size = random.choice([5, 7, 9])
            angle = random.randint(0, 180)
            img = apply_motion_blur(img, size=size, angle=angle)
    return img

# ========== SQUARE GENERATOR ==========
def draw_square(img, cname, classid):
    bgr = colors[cname][0]
    size = random.randint(40, 90)
    cx, cy = random.randint(50, 170), random.randint(50, 170)
    x_squeeze = random.uniform(0.85, 1.0)
    y_squeeze = random.uniform(0.85, 1.0)
    half = size / 2

    pts = np.array([
        [-half * x_squeeze, -half * y_squeeze],
        [half * x_squeeze, -half * y_squeeze],
        [half, half],
        [-half, half]
    ], dtype=np.float32)

    if random.random() < 0.5:
        shear = random.uniform(-0.15, 0.15)
        pts = np.dot(pts, np.array([[1, shear], [0, 1]]))

    pts += np.array([cx, cy])
    pts = pts.astype(np.int32)
    cv2.fillConvexPoly(img, pts, bgr)

    x_min, y_min = np.min(pts, axis=0)
    x_max, y_max = np.max(pts, axis=0)
    xc = ((x_min + x_max) / 2) / img_size
    yc = ((y_min + y_max) / 2) / img_size
    w = (x_max - x_min) / img_size
    h = (y_max - y_min) / img_size
    return f"{classid} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}"

# ========== MAIN LOOP ==========
print("Generating messy synthetic dataset...")
for i in tqdm(range(total_images)):
    img = make_background()
    label_lines = []
    for cname in random.sample(list(colors.keys()), random.randint(min_squares, max_squares)):
        label = draw_square(img, cname, colors[cname][1])
        label_lines.append(label)

    img = apply_brightness_contrast(img)
    img = apply_blur(img)

    fname = f"messy_{i:05d}.jpg"
    cv2.imwrite(os.path.join(img_dir, fname), img)
    with open(os.path.join(lbl_dir, fname.replace(".jpg", ".txt")), "w") as f:
        f.write("\n".join(label_lines))

print("✅ Done! Dataset saved to:", output_dir)


Generating messy synthetic dataset...


100%|██████████| 1000/1000 [00:05<00:00, 180.17it/s]

✅ Done! Dataset saved to: synthetic_squares_messy


In [7]:
import os
import cv2
import numpy as np
import random
from PIL import Image, ImageEnhance
from tqdm import tqdm

# ==== CONFIG ====
output_dir = "synthetic_squares_final"
img_dir = os.path.join(output_dir, "images")
lbl_dir = os.path.join(output_dir, "labels")
os.makedirs(img_dir, exist_ok=True)
os.makedirs(lbl_dir, exist_ok=True)

img_size = 224
total_images = 10000
min_squares = 1
max_squares = 3
blur_prob = 0.5
brightness_jitter = 0.4

colors = {
    "blue":   ((255, 30, 30), 0),
    "red":    ((30, 30, 255), 1),
    "orange": ((0, 165, 255), 2),
    "yellow": ((0, 255, 255), 3),
    "purple": ((200, 80, 200), 4),
    "brown":  ((139, 69, 19), 5),
    "gray":   ((128, 128, 128), 6),
    "black":  ((0, 0, 0), 7),
    "pink":   ((240, 110, 180), 8),
    "white":  ((255, 255, 255), 9),
    "green":  ((65, 180, 80), 10),
}
class_names = list(colors.keys())

# Clutter colors EXCLUDING all target colors (i.e., no white, gray, brown, yellow, orange, blue, red, pink, purple, black, green)
# Only "distractor" colors
CLUTTER_COLORS = [
    ((0, 255, 255), 0.07),    # Cyan
    ((255, 20, 147), 0.06),   # Deep Pink
    ((100, 149, 237), 0.08),  # Cornflower Blue
    ((34, 139, 34), 0.06),    # Forest Green
    ((255, 215, 0), 0.07),    # Gold
    ((255, 140, 0), 0.08),    # Dark Orange
    ((0, 206, 209), 0.07),    # Dark Turquoise
    ((186, 85, 211), 0.05),   # Medium Orchid
    ((255, 0, 255), 0.07),    # Magenta
    ((70, 130, 180), 0.07),   # Steel Blue
    ((173, 255, 47), 0.08),   # Green Yellow
    ((205, 133, 63), 0.07),   # Peru
    # Add more if you want!
]
CLUTTER_RGBS, CLUTTER_PROBS = zip(*CLUTTER_COLORS)
def sample_clutter_color():
    idx = random.choices(range(len(CLUTTER_RGBS)), weights=CLUTTER_PROBS, k=1)[0]
    return CLUTTER_RGBS[idx]

def make_background():
    bg = np.full((img_size, img_size, 3), random.randint(100, 200), dtype=np.uint8)
    for _ in range(random.randint(5, 20)):
        color = tuple(int(x) for x in sample_clutter_color())
        center = (random.randint(0, img_size), random.randint(0, img_size))
        radius = random.randint(10, 50)
        cv2.circle(bg, center, radius, color, -1)
    for _ in range(random.randint(3, 10)):
        pt1 = (random.randint(0, img_size), random.randint(0, img_size))
        pt2 = (random.randint(0, img_size), random.randint(0, img_size))
        color = tuple(int(x) for x in sample_clutter_color())
        thickness = random.choice([1, 2])
        cv2.line(bg, pt1, pt2, color, thickness)
    for _ in range(random.randint(3, 8)):
        top_left = (random.randint(0, img_size - 30), random.randint(0, img_size - 30))
        w, h = random.randint(10, 50), random.randint(10, 50)
        color = tuple(int(x) for x in sample_clutter_color())
        cv2.rectangle(bg, top_left, (top_left[0] + w, top_left[1] + h), color, -1)
    # Vignette effect
    Y, X = np.ogrid[:img_size, :img_size]
    center = (img_size / 2, img_size / 2)
    dist_from_center = np.sqrt((X - center[0])**2 + (Y - center[1])**2)
    vignette = 1 - 0.6 * (dist_from_center / dist_from_center.max())
    for i in range(3):
        bg[..., i] = np.clip(bg[..., i] * vignette, 0, 255)
    # Camera noise
    noise = np.random.normal(0, 14, (img_size, img_size, 3))
    bg = np.clip(bg + noise, 0, 255).astype(np.uint8)
    return bg

def apply_brightness_contrast(img):
    img = Image.fromarray(img)
    img = ImageEnhance.Brightness(img).enhance(random.uniform(0.7, 1.3))
    img = ImageEnhance.Contrast(img).enhance(random.uniform(0.7, 1.3))
    return np.array(img)

def apply_motion_blur(image, size=5, angle=0):
    kernel = np.zeros((size, size))
    kernel[int((size - 1) / 2), :] = np.ones(size)
    M = cv2.getRotationMatrix2D((size / 2 - 0.5, size / 2 - 0.5), angle, 1.0)
    kernel = cv2.warpAffine(kernel, M, (size, size))
    kernel /= np.sum(kernel)
    return cv2.filter2D(image, -1, kernel)

def apply_blur(img):
    if random.random() < blur_prob:
        ksize = random.choice([3, 5, 7])
        img = cv2.GaussianBlur(img, (ksize, ksize), 0)
        if random.random() < 0.3:
            size = random.choice([5, 7, 9])
            angle = random.randint(0, 180)
            img = apply_motion_blur(img, size=size, angle=angle)
    return img

def add_tape_reflection(img, pts):
    if random.random() < 0.3:
        x_min, y_min = np.min(pts, axis=0)
        x_max, y_max = np.max(pts, axis=0)
        box_w, box_h = x_max - x_min, y_max - y_min
        tape_w = int(box_w * random.uniform(0.1, 0.5))
        tape_h = int(box_h * random.uniform(0.06, 0.2))
        angle = random.uniform(-30, 30)
        center = (
            int((x_min + x_max) / 2 + random.uniform(-0.1, 0.1) * box_w),
            int((y_min + y_max) / 2 + random.uniform(-0.1, 0.1) * box_h)
        )
        tape = np.zeros_like(img)
        rect = (
            (center[0], center[1]),
            (tape_w, tape_h),
            angle
        )
        box = cv2.boxPoints(rect).astype(int)
        cv2.fillConvexPoly(tape, box, (255, 255, 255))
        tape = cv2.GaussianBlur(tape, (11, 11), 0)
        alpha = random.uniform(0.45, 0.8)
        mask = (tape > 0).astype(np.float32)
        img = np.clip(img * (1 - mask * alpha) + tape * alpha, 0, 255).astype(np.uint8)
    return img

def draw_square(img, cname, classid):
    bgr = colors[cname][0]
    size = random.randint(40, 90)
    cx, cy = random.randint(50, 170), random.randint(50, 170)
    x_squeeze = random.uniform(0.85, 1.0)
    y_squeeze = random.uniform(0.85, 1.0)
    half = size / 2
    # --- Rounded corner logic ---
    radius = int(size * random.uniform(0.06, 0.18)) if random.random() < 0.5 else 0
    pts = np.array([
        [-half * x_squeeze, -half * y_squeeze],
        [half * x_squeeze, -half * y_squeeze],
        [half, half],
        [-half, half]
    ], dtype=np.float32)
    if random.random() < 0.5:
        shear = random.uniform(-0.15, 0.15)
        pts = np.dot(pts, np.array([[1, shear], [0, 1]]))
    pts += np.array([cx, cy])
    pts = pts.astype(np.int32)

    # Draw rounded or regular square
    if radius > 0:
        rect = cv2.boundingRect(pts)
        mask = np.zeros(img.shape, dtype=np.uint8)
        x, y, w, h = rect
        cv2.rectangle(mask, (x, y), (x+w, y+h), bgr, -1)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = cv2.blur(mask, (radius*2+1, radius*2+1))
        shape = np.zeros(img.shape, dtype=np.uint8)
        cv2.fillConvexPoly(shape, pts, (255,255,255))
        mask = cv2.bitwise_and(mask, cv2.cvtColor(shape, cv2.COLOR_BGR2GRAY))
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        img = np.where(mask > 0, np.array(bgr, dtype=np.uint8), img)
    else:
        cv2.fillConvexPoly(img, pts, bgr)

    # Simulate tape reflection
    img = add_tape_reflection(img, pts)

    x_min, y_min = np.min(pts, axis=0)
    x_max, y_max = np.max(pts, axis=0)
    xc = ((x_min + x_max) / 2) / img_size
    yc = ((y_min + y_max) / 2) / img_size
    w = (x_max - x_min) / img_size
    h = (y_max - y_min) / img_size
    return f"{classid} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}"

# Optionally: add partial occlusion, fake folds, or scribbles for even more realism!
def add_occlusion(img):
    if random.random() < 0.1:
        x1, y1 = random.randint(0, img_size-40), random.randint(0, img_size-40)
        w, h = random.randint(15, 45), random.randint(15, 45)
        color = tuple(int(x) for x in sample_clutter_color())
        alpha = random.uniform(0.35, 0.8)
        overlay = img.copy()
        cv2.rectangle(overlay, (x1, y1), (x1 + w, y1 + h), color, -1)
        img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)
    return img

def sample_clutter_color():
    idx = random.choices(range(len(CLUTTER_RGBS)), weights=CLUTTER_PROBS, k=1)[0]
    return CLUTTER_RGBS[idx]

# ========== MAIN LOOP ==========
print("Generating final robust synthetic dataset...")
for i in tqdm(range(total_images)):
    img = make_background()
    label_lines = []
    for cname in random.sample(list(colors.keys()), random.randint(min_squares, max_squares)):
        label = draw_square(img, cname, colors[cname][1])
        label_lines.append(label)
    img = apply_brightness_contrast(img)
    img = apply_blur(img)
    img = add_occlusion(img)
    fname = f"final_{i:05d}.jpg"
    cv2.imwrite(os.path.join(img_dir, fname), img)
    with open(os.path.join(lbl_dir, fname.replace(".jpg", ".txt")), "w") as f:
        f.write("\n".join(label_lines))
print("✅ Done! Dataset saved to:", output_dir)


Generating final robust synthetic dataset...


100%|██████████| 10000/10000 [01:21<00:00, 121.98it/s]

✅ Done! Dataset saved to: synthetic_squares_final


## Extreme Noise

In [8]:
import os
import cv2
import numpy as np
import random
from PIL import Image, ImageEnhance
from tqdm import tqdm

# ==== CONFIG ====
output_dir = "synthetic_squares_extreme"
img_dir = os.path.join(output_dir, "images")
lbl_dir = os.path.join(output_dir, "labels")
os.makedirs(img_dir, exist_ok=True)
os.makedirs(lbl_dir, exist_ok=True)

img_size = 224
total_images = 10000
min_squares = 1
max_squares = 3
blur_prob = 0.7  # more aggressive
brightness_jitter = 0.6

colors = {
    "blue":   ((255, 30, 30), 0),
    "red":    ((30, 30, 255), 1),
    "orange": ((0, 165, 255), 2),
    "yellow": ((0, 255, 255), 3),
    "purple": ((200, 80, 200), 4),
    "brown":  ((139, 69, 19), 5),
    "gray":   ((128, 128, 128), 6),
    "black":  ((0, 0, 0), 7),
    "pink":   ((240, 110, 180), 8),
    "white":  ((255, 255, 255), 9),
    "green":  ((65, 180, 80), 10),
}
class_names = list(colors.keys())

# Extreme distractor colors (never target class colors)
CLUTTER_COLORS = [
    ((0, 255, 255), 0.07),    # Cyan
    ((255, 20, 147), 0.06),   # Deep Pink
    ((100, 149, 237), 0.08),  # Cornflower Blue
    ((34, 139, 34), 0.06),    # Forest Green
    ((255, 215, 0), 0.07),    # Gold
    ((255, 140, 0), 0.08),    # Dark Orange
    ((0, 206, 209), 0.07),    # Dark Turquoise
    ((186, 85, 211), 0.05),   # Medium Orchid
    ((255, 0, 255), 0.07),    # Magenta
    ((70, 130, 180), 0.07),   # Steel Blue
    ((173, 255, 47), 0.08),   # Green Yellow
    ((205, 133, 63), 0.07),   # Peru
]
CLUTTER_RGBS, CLUTTER_PROBS = zip(*CLUTTER_COLORS)
def sample_clutter_color():
    idx = random.choices(range(len(CLUTTER_RGBS)), weights=CLUTTER_PROBS, k=1)[0]
    return CLUTTER_RGBS[idx]

def add_gradient(bg):
    # Choose up to 3 random gradient directions/colors
    for _ in range(random.randint(1, 3)):
        color1 = np.array(sample_clutter_color(), dtype=np.float32)
        color2 = np.array(sample_clutter_color(), dtype=np.float32)
        direction = random.choice(['horizontal', 'vertical', 'diag1', 'diag2'])
        gradient = np.zeros_like(bg, dtype=np.float32)
        if direction == 'horizontal':
            for i in range(img_size):
                alpha = i / img_size
                gradient[:, i] = (1 - alpha) * color1 + alpha * color2
        elif direction == 'vertical':
            for i in range(img_size):
                alpha = i / img_size
                gradient[i, :] = (1 - alpha) * color1 + alpha * color2
        elif direction == 'diag1':
            for i in range(img_size):
                for j in range(img_size):
                    alpha = (i + j) / (2 * img_size)
                    gradient[i, j] = (1 - alpha) * color1 + alpha * color2
        elif direction == 'diag2':
            for i in range(img_size):
                for j in range(img_size):
                    alpha = abs(i - j) / img_size
                    gradient[i, j] = (1 - alpha) * color1 + alpha * color2
        bg = cv2.addWeighted(bg.astype(np.float32), 0.7, gradient, 0.3, 0).astype(np.uint8)
    return bg

def add_pen_marks(img):
    # Simulate pen marks/scribbles on the whole image
    for _ in range(random.randint(4, 10)):
        color = tuple(int(x) for x in sample_clutter_color())
        thickness = random.choice([1, 2, 3])
        mark_type = random.choice(['line', 'curve', 'ellipse', 'zigzag'])
        if mark_type == 'line':
            pt1 = (random.randint(0, img_size), random.randint(0, img_size))
            pt2 = (random.randint(0, img_size), random.randint(0, img_size))
            cv2.line(img, pt1, pt2, color, thickness)
        elif mark_type == 'curve':
            pts = np.array([
                [random.randint(0, img_size), random.randint(0, img_size)],
                [random.randint(0, img_size), random.randint(0, img_size)],
                [random.randint(0, img_size), random.randint(0, img_size)],
            ], np.int32).reshape((-1, 1, 2))
            cv2.polylines(img, [pts], False, color, thickness)
        elif mark_type == 'ellipse':
            center = (random.randint(0, img_size), random.randint(0, img_size))
            axes = (random.randint(10, 80), random.randint(10, 80))
            angle = random.uniform(0, 360)
            start = random.randint(0, 180)
            end = start + random.randint(30, 180)
            cv2.ellipse(img, center, axes, angle, start, end, color, thickness)
        elif mark_type == 'zigzag':
            pt = [random.randint(0, img_size), random.randint(0, img_size)]
            for _ in range(random.randint(3, 7)):
                pt2 = [pt[0] + random.randint(-40, 40), pt[1] + random.randint(-40, 40)]
                pt2[0] = np.clip(pt2[0], 0, img_size-1)
                pt2[1] = np.clip(pt2[1], 0, img_size-1)
                cv2.line(img, tuple(pt), tuple(pt2), color, thickness)
                pt = pt2
    return img

def make_background():
    bg = np.full((img_size, img_size, 3), random.randint(100, 200), dtype=np.uint8)
    bg = add_gradient(bg)
    for _ in range(random.randint(8, 25)):
        color = tuple(int(x) for x in sample_clutter_color())
        center = (random.randint(0, img_size), random.randint(0, img_size))
        radius = random.randint(10, 50)
        cv2.circle(bg, center, radius, color, -1)
    for _ in range(random.randint(5, 20)):
        pt1 = (random.randint(0, img_size), random.randint(0, img_size))
        pt2 = (random.randint(0, img_size), random.randint(0, img_size))
        color = tuple(int(x) for x in sample_clutter_color())
        thickness = random.choice([1, 2, 3])
        cv2.line(bg, pt1, pt2, color, thickness)
    for _ in range(random.randint(6, 12)):
        top_left = (random.randint(0, img_size - 30), random.randint(0, img_size - 30))
        w, h = random.randint(10, 60), random.randint(10, 60)
        color = tuple(int(x) for x in sample_clutter_color())
        cv2.rectangle(bg, top_left, (top_left[0] + w, top_left[1] + h), color, -1)
    bg = add_pen_marks(bg)
    # Vignette
    Y, X = np.ogrid[:img_size, :img_size]
    center = (img_size / 2, img_size / 2)
    dist_from_center = np.sqrt((X - center[0])**2 + (Y - center[1])**2)
    vignette = 1 - 0.7 * (dist_from_center / dist_from_center.max())
    for i in range(3):
        bg[..., i] = np.clip(bg[..., i] * vignette, 0, 255)
    # Camera noise
    noise = np.random.normal(0, 20, (img_size, img_size, 3))
    bg = np.clip(bg + noise, 0, 255).astype(np.uint8)
    # Occasionally camera "overexpose" (white blowout) somewhere
    if random.random() < 0.08:
        x0, y0 = random.randint(0, img_size-70), random.randint(0, img_size-70)
        w, h = random.randint(30, 80), random.randint(30, 80)
        overlay = bg.copy()
        cv2.rectangle(overlay, (x0, y0), (x0+w, y0+h), (255,255,255), -1)
        alpha = random.uniform(0.4, 0.8)
        bg = cv2.addWeighted(overlay, alpha, bg, 1-alpha, 0)
    return bg

def apply_brightness_contrast(img):
    img = Image.fromarray(img)
    img = ImageEnhance.Brightness(img).enhance(random.uniform(0.55, 1.5))
    img = ImageEnhance.Contrast(img).enhance(random.uniform(0.5, 1.6))
    return np.array(img)

def apply_motion_blur(image, size=5, angle=0):
    kernel = np.zeros((size, size))
    kernel[int((size - 1) / 2), :] = np.ones(size)
    M = cv2.getRotationMatrix2D((size / 2 - 0.5, size / 2 - 0.5), angle, 1.0)
    kernel = cv2.warpAffine(kernel, M, (size, size))
    kernel /= np.sum(kernel)
    return cv2.filter2D(image, -1, kernel)

def apply_blur(img):
    if random.random() < blur_prob:
        ksize = random.choice([5, 7, 9])
        img = cv2.GaussianBlur(img, (ksize, ksize), 0)
        if random.random() < 0.5:
            size = random.choice([7, 9, 15])
            angle = random.randint(0, 180)
            img = apply_motion_blur(img, size=size, angle=angle)
    return img

def add_tape_reflection(img, pts):
    # Strong white tape glare simulation
    if random.random() < 0.37:
        x_min, y_min = np.min(pts, axis=0)
        x_max, y_max = np.max(pts, axis=0)
        box_w, box_h = x_max - x_min, y_max - y_min
        tape_w = int(box_w * random.uniform(0.13, 0.5))
        tape_h = int(box_h * random.uniform(0.06, 0.23))
        angle = random.uniform(-40, 40)
        center = (
            int((x_min + x_max) / 2 + random.uniform(-0.18, 0.18) * box_w),
            int((y_min + y_max) / 2 + random.uniform(-0.18, 0.18) * box_h)
        )
        tape = np.zeros_like(img)
        rect = (
            (center[0], center[1]),
            (tape_w, tape_h),
            angle
        )
        box = cv2.boxPoints(rect).astype(int)
        cv2.fillConvexPoly(tape, box, (255, 255, 255))
        tape = cv2.GaussianBlur(tape, (17, 17), 0)
        alpha = random.uniform(0.58, 0.92)
        mask = (tape > 0).astype(np.float32)
        img = np.clip(img * (1 - mask * alpha) + tape * alpha, 0, 255).astype(np.uint8)
    # Even more extreme: sometimes blow out whole square to nearly white
    if random.random() < 0.06:
        cv2.fillConvexPoly(img, pts, (255, 255, 255))
        img = cv2.GaussianBlur(img, (13, 13), 0)
    return img

def draw_square(img, cname, classid):
    bgr = colors[cname][0]
    size = random.randint(35, 95)
    cx, cy = random.randint(50, 170), random.randint(50, 170)
    x_squeeze = random.uniform(0.82, 1.0)
    y_squeeze = random.uniform(0.82, 1.0)
    half = size / 2
    # --- Rounded corners ---
    radius = int(size * random.uniform(0.07, 0.22)) if random.random() < 0.7 else 0
    pts = np.array([
        [-half * x_squeeze, -half * y_squeeze],
        [half * x_squeeze, -half * y_squeeze],
        [half, half],
        [-half, half]
    ], dtype=np.float32)
    if random.random() < 0.6:
        shear = random.uniform(-0.19, 0.19)
        pts = np.dot(pts, np.array([[1, shear], [0, 1]]))
    pts += np.array([cx, cy])
    pts = pts.astype(np.int32)
    # Draw rounded or regular square
    if radius > 0:
        rect = cv2.boundingRect(pts)
        mask = np.zeros(img.shape, dtype=np.uint8)
        x, y, w, h = rect
        cv2.rectangle(mask, (x, y), (x+w, y+h), bgr, -1)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = cv2.blur(mask, (radius*2+1, radius*2+1))
        shape = np.zeros(img.shape, dtype=np.uint8)
        cv2.fillConvexPoly(shape, pts, (255,255,255))
        mask = cv2.bitwise_and(mask, cv2.cvtColor(shape, cv2.COLOR_BGR2GRAY))
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        img = np.where(mask > 0, np.array(bgr, dtype=np.uint8), img)
    else:
        cv2.fillConvexPoly(img, pts, bgr)
    img = add_tape_reflection(img, pts)
    x_min, y_min = np.min(pts, axis=0)
    x_max, y_max = np.max(pts, axis=0)
    xc = ((x_min + x_max) / 2) / img_size
    yc = ((y_min + y_max) / 2) / img_size
    w = (x_max - x_min) / img_size
    h = (y_max - y_min) / img_size
    return f"{classid} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}"

def add_occlusion(img):
    if random.random() < 0.18:
        x1, y1 = random.randint(0, img_size-40), random.randint(0, img_size-40)
        w, h = random.randint(15, 45), random.randint(15, 45)
        color = tuple(int(x) for x in sample_clutter_color())
        alpha = random.uniform(0.40, 0.9)
        overlay = img.copy()
        cv2.rectangle(overlay, (x1, y1), (x1 + w, y1 + h), color, -1)
        img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)
    return img

print("Generating EXTREME synthetic dataset...")
for i in tqdm(range(total_images)):
    img = make_background()
    label_lines = []
    for cname in random.sample(list(colors.keys()), random.randint(min_squares, max_squares)):
        label = draw_square(img, cname, colors[cname][1])
        label_lines.append(label)
    img = apply_brightness_contrast(img)
    img = apply_blur(img)
    img = add_occlusion(img)
    # Final global whiteout flare (simulate sun/flash)
    if random.random() < 0.04:
        mask = np.zeros((img_size, img_size, 3), dtype=np.uint8)
        x, y = random.randint(40, img_size-40), random.randint(40, img_size-40)
        cv2.circle(mask, (x, y), random.randint(25, 60), (255,255,255), -1)
        mask = cv2.GaussianBlur(mask, (21, 21), 0)
        alpha = random.uniform(0.3, 0.7)
        img = np.clip(img * (1-alpha) + mask * alpha, 0, 255).astype(np.uint8)
    fname = f"extreme_{i:05d}.jpg"
    cv2.imwrite(os.path.join(img_dir, fname), img)
    with open(os.path.join(lbl_dir, fname.replace(".jpg", ".txt")), "w") as f:
        f.write("\n".join(label_lines))
print("✅ Done! Extreme dataset saved to:", output_dir)


Generating EXTREME synthetic dataset...


100%|██████████| 10000/10000 [24:21<00:00,  6.84it/s]

✅ Done! Extreme dataset saved to: synthetic_squares_extreme


### Extreme Noise debugging Dataset generator


In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
EXTREME synthetic square generator (every rectangle boxed)
----------------------------------------------------------
 • Creates training images + YOLO labels.
 • Makes debug images with perfectly aligned green boxes.
 • *No* square is skipped – every one receives a box.
"""

import os, cv2, random, numpy as np
from PIL import Image, ImageEnhance
from tqdm import tqdm

# ──────────────────────────────  OUTPUT FOLDERS  ──────────────────────────────
out_root = "synthetic_squares_extreme_fullbox"
img_dir  = os.path.join(out_root, "images")       # training images
lbl_dir  = os.path.join(out_root, "labels")       # YOLO txt files
vis_dir  = os.path.join(out_root, "images_vis")   # debug images
for d in (img_dir, lbl_dir, vis_dir):
    os.makedirs(d, exist_ok=True)

# ───────────────────────────────  PARAMETERS  ────────────────────────────────
img_size      = 224
N_IMAGES      = 10_000
MIN_SQUARES   = 1
MAX_SQUARES   = 3
BLUR_PROB     = 0.70

colors = {                     #  BGR  , class-id
    "blue"  : ((255,  30,  30), 0),
    "red"   : (( 30,  30, 255), 1),
    "orange": ((  0, 165, 255), 2),
    "yellow": ((  0, 255, 255), 3),
    "purple": ((200,  80, 200), 4),
    "brown" : ((139,  69,  19), 5),
    "gray"  : ((128, 128, 128), 6),
    "black" : ((  0,   0,   0), 7),
    "pink"  : ((240, 110, 180), 8),
    "white" : ((255, 255, 255), 9),
    "green" : (( 65, 180,  80),10),
}

# distractor colours never used for target squares
CLUTTER_RGBS, CLUTTER_PROBS = zip(*[
    ((  0,255,255),0.07),((255, 20,147),0.06),((100,149,237),0.08),
    (( 34,139, 34),0.06),((255,215,  0),0.07),((255,140,  0),0.08),
    ((  0,206,209),0.07),((186, 85,211),0.05),((255,  0,255),0.07),
    (( 70,130,180),0.07),((173,255, 47),0.08),((205,133, 63),0.07)
])
rand_colour = lambda: random.choices(CLUTTER_RGBS, CLUTTER_PROBS, k=1)[0]

# ─────────────────────────────── UTILITIES ───────────────────────────────────
def make_background():
    bg = np.full((img_size,img_size,3), random.randint(100,200), np.uint8)

    # ––– gradients –––
    for _ in range(random.randint(1,3)):
        c1, c2 = map(np.float32, (rand_colour(), rand_colour()))
        grad   = np.zeros_like(bg, np.float32)
        mode   = random.choice(("hor","ver","d1","d2"))
        for y in range(img_size):
            for x in range(img_size):
                a = (
                    x/img_size             if mode=="hor" else
                    y/img_size             if mode=="ver" else
                    (x+y)/(2*img_size)     if mode=="d1" else
                    abs(x-y)/img_size
                )
                grad[y,x] = (1-a)*c1 + a*c2
        bg = cv2.addWeighted(bg,0.7, grad.astype(np.uint8),0.3,0)

    # ––– clutter –––
    for _ in range(random.randint(8,25)):
        cv2.circle(bg,(random.randint(0,img_size),random.randint(0,img_size)),
                   random.randint(10,50), rand_colour(), -1)
    for _ in range(random.randint(5,20)):
        cv2.line(bg,(random.randint(0,img_size),random.randint(0,img_size)),
                 (random.randint(0,img_size),random.randint(0,img_size)),
                 rand_colour(), random.choice((1,2,3)))
    for _ in range(random.randint(6,12)):
        tl = (random.randint(0,img_size-30), random.randint(0,img_size-30))
        w,h = random.randint(10,60), random.randint(10,60)
        cv2.rectangle(bg, tl, (tl[0]+w,tl[1]+h), rand_colour(), -1)

    # vignette + noise
    Y,X = np.ogrid[:img_size,:img_size]
    vign = 1 - 0.7*np.sqrt((X-img_size/2)**2 + (Y-img_size/2)**2)/(np.sqrt(2)*img_size/2)
    bg   = np.clip(bg*vign[...,None],0,255).astype(np.uint8)
    bg   = np.clip(bg + np.random.normal(0,20,(img_size,img_size,3)),0,255).astype(np.uint8)
    return bg

def apply_brightness_contrast(im):
    pil = Image.fromarray(im)
    pil = ImageEnhance.Brightness(pil).enhance(random.uniform(0.55,1.5))
    pil = ImageEnhance.Contrast (pil).enhance(random.uniform(0.5 ,1.6))
    return np.array(pil)

def apply_blur(im):
    if random.random() < BLUR_PROB:
        im = cv2.GaussianBlur(im, (random.choice((5,7,9)),)*2, 0)
        if random.random() < 0.5:                     # optional motion blur
            k = random.choice((7,9,15)); ang = random.randint(0,180)
            ker = np.zeros((k,k)); ker[k//2]=1
            M   = cv2.getRotationMatrix2D((k/2-0.5,k/2-0.5),ang,1)
            ker = cv2.warpAffine(ker,M,(k,k)); ker /= ker.sum()
            im  = cv2.filter2D(im,-1,ker)
    return im

# ───────────────────────────  DRAW ONE SQUARE  ──────────────────────────────
def draw_square(canvas, colour_name):
    """Draw square with random squeeze & shear. Return YOLO label + bbox."""
    bgr, class_id = colors[colour_name]
    size          = random.randint(35,95)
    cx,cy         = random.randint(50,170), random.randint(50,170)
    xs,ys         = random.uniform(0.82,1.0), random.uniform(0.82,1.0)
    hlf           = size/2

    pts = np.array([
        [-hlf*xs,-hlf*ys], [ hlf*xs,-hlf*ys],
        [ hlf    , hlf   ], [-hlf   , hlf   ]
    ], np.float32)

    if random.random()<0.6:
        shear = random.uniform(-0.19,0.19)
        pts   = pts @ np.array([[1,shear],[0,1]])

    pts += (cx,cy); pts = pts.astype(np.int32)
    cv2.fillConvexPoly(canvas, pts, bgr)

    x,y,w,h = cv2.boundingRect(pts)
    xc,yc   = (x+w/2)/img_size, (y+h/2)/img_size
    label   = f"{class_id} {xc:.6f} {yc:.6f} {w/img_size:.6f} {h/img_size:.6f}"
    return label, (x,y,x+w,y+h)

# ───────────────────────────────  MAIN LOOP  ────────────────────────────────
print("Generating EXTREME set (every square boxed)…")
for i in tqdm(range(N_IMAGES)):
    img   = make_background()
    lines = []          # YOLO txt lines
    boxes = []          # (x1,y1,x2,y2,class_id) for debug-draw

    for cname in random.sample(list(colors), random.randint(MIN_SQUARES,MAX_SQUARES)):
        lbl, (x1,y1,x2,y2) = draw_square(img, cname)
        lines.append(lbl)
        boxes.append((x1,y1,x2,y2,lbl.split()[0]))

    # pixel-only augmentations
    img = apply_brightness_contrast(img)
    img = apply_blur(img)

    # copy for visualisation AFTER augmentations
    vis = img.copy()
    for x1,y1,x2,y2,cid in boxes:
        cv2.rectangle(vis,(x1,y1),(x2,y2),(0,255,0),2)
        cv2.putText(vis,cid,(x1,max(0,y1-4)),
                    cv2.FONT_HERSHEY_SIMPLEX,0.4,(0,255,0),1,cv2.LINE_AA)

    name = f"extreme_{i:05d}.jpg"
    cv2.imwrite(os.path.join(img_dir ,name), img)
    cv2.imwrite(os.path.join(vis_dir,name), vis)
    with open(os.path.join(lbl_dir,name.replace(".jpg",".txt")),'w') as f:
        f.write("\n".join(lines))

print("✅  Finished.  Data saved under:", out_root)


Generating EXTREME set (every square boxed)…


100%|██████████| 10000/10000 [42:09<00:00,  3.95it/s] 

✅  Finished.  Data saved under: synthetic_squares_extreme_fullbox


In [7]:
import os
import shutil
import random

# === CONFIG ===
root = r"C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme_fullbox"
img_folder = os.path.join(root, "images")
lbl_folder = os.path.join(root, "labels")
train_ratio = 0.85  # 85% train, 15% val

# === Ensure output folders exist ===
def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

for split in ['train', 'val']:
    ensure_dir(os.path.join(img_folder, split))
    ensure_dir(os.path.join(lbl_folder, split))

# === List all image files ===
all_imgs = [f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]
random.shuffle(all_imgs)

split_idx = int(len(all_imgs) * train_ratio)
train_imgs = all_imgs[:split_idx]
val_imgs = all_imgs[split_idx:]

def move_split(imgs, split):
    for fname in imgs:
        src_img = os.path.join(img_folder, fname)
        dst_img = os.path.join(img_folder, split, fname)
        shutil.move(src_img, dst_img)
        lbl_name = os.path.splitext(fname)[0] + '.txt'
        src_lbl = os.path.join(lbl_folder, lbl_name)
        dst_lbl = os.path.join(lbl_folder, split, lbl_name)
        if os.path.exists(src_lbl):
            shutil.move(src_lbl, dst_lbl)
        else:
            print(f"Warning: missing label for {fname}")

move_split(train_imgs, 'train')
move_split(val_imgs, 'val')

print(f"Split {len(all_imgs)} images into {len(train_imgs)} train and {len(val_imgs)} val.")

# === Write YOLOv8 YAML config ===
yaml_text = f"""
train: {img_folder.replace(os.sep, '/')}/train
val: {img_folder.replace(os.sep, '/')}/val
nc: 11
names: ["blue", "red", "orange", "yellow", "purple", "brown", "gray", "black", "pink", "white", "green"]
"""

yaml_path = os.path.join(root, "synthetic_extreme.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_text.strip())

print(f"YAML config written to {yaml_path}")

# === READY for YOLOv8 training! ===


Split 10000 images into 8500 train and 1500 val.
YAML config written to C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme_fullbox\synthetic_extreme.yaml


In [ ]:
from ultralytics import YOLO
import cv2
import os
from tqdm import tqdm

# ==== TRAINING ====
yaml_path = r"C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme_fullbox\synthetic_extreme.yaml"
model = YOLO('yolov8n.pt')

results = model.train(
    data=yaml_path,
    epochs=200,
    imgsz=224,
    patience=30,
    batch=24,
    device=0,
    project='runs',
    name='synthetic_extreme'
)
metrics = model.val()

# ==== PREDICT AND VISUALIZE ====
# Input and output folders
in_folder = r"C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train"
out_folder = r"C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\pred_vis"
os.makedirs(out_folder, exist_ok=True)

# Load best trained weights
model = YOLO('runs/synthetic_extreme/weights/best.pt')

# Class names
class_names = ["blue", "red", "orange", "yellow", "purple", "brown", "gray", "black", "pink", "white", "green"]

for fname in tqdm(os.listdir(in_folder)):
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    img_path = os.path.join(in_folder, fname)
    img = cv2.imread(img_path)
    results = model(img_path)
    for box in results[0].boxes:
        x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0]]
        class_id = int(box.cls[0])
        conf = float(box.conf[0])
        label = f"{class_names[class_id]} {conf:.2f}"
        color = (0, 255, 0)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.65, color, 2, cv2.LINE_AA)
    out_path = os.path.join(out_folder, fname)
    cv2.imwrite(out_path, img)

print(f"All predictions with bounding boxes saved to {out_folder}")


Ultralytics 8.3.141  Python-3.9.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme\synthetic_extreme.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=synthetic_extreme2, nbs=64, nms=False, opset=None, optimize=F

100%|██████████| 5.35M/5.35M [00:00<00:00, 8.25MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.10.0 ms, read: 52.127.1 MB/s, size: 19.3 KB)


train: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme\labels\train.cache... 8500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8500/8500 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 42.316.9 MB/s, size: 14.8 KB)


...


image 1/1 C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\yellow_20250521-111419-787989.jpg: 224x224 1 white, 4.7ms
Speed: 0.5ms preprocess, 4.7ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\yellow_20250521-111420-378811.jpg: 224x224 (no detections), 5.1ms
Speed: 0.5ms preprocess, 5.1ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\yellow_20250521-111420-987267.jpg: 224x224 (no detections), 5.0ms
Speed: 0.5ms preprocess, 5.0ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\yellow_20250521-111421-271372.jpg: 224x224 (no detections), 5.2ms
Speed: 0.5ms preprocess, 5.2ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 C:\Users\fredr\BTH\Roboti

 98%|█████████▊| 59/60 [00:00<00:00, 76.98it/s]


image 1/1 C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\yellow_20250521-111424-850402.jpg: 224x224 1 yellow, 1 white, 5.1ms
Speed: 0.6ms preprocess, 5.1ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 224)


100%|██████████| 60/60 [00:00<00:00, 61.29it/s]

All predictions with bounding boxes saved to C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\pred_vis


In [1]:
import torch
print("CUDA available?", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current device:", torch.cuda.current_device(), torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("WARNING: No GPU found. Training will be very slow.")

from ultralytics import YOLO

yaml_path = r"C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme_fullbox\synthetic_extreme.yaml"

# Use YOLOv8n for Jetbot compatibility!
model = YOLO('yolov8n.pt')

results = model.train(
    data=yaml_path,
    epochs=300,
    imgsz=224,
    patience=50,
    batch=32,               # Try 32, or lower if OOM
    lr0=0.001,
    optimizer='SGD',
    device=0,               # GPU
    project='runs',
    name='synthetic_extreme_yolov8n',
    cos_lr=True             # Cosine LR
)

metrics = model.val()


CUDA available? True
CUDA device count: 1
Current device: 0 NVIDIA GeForce RTX 3070 Ti
New https://pypi.org/project/ultralytics/8.3.142 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.141  Python-3.9.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme_fullbox\synthetic_extreme.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.

train: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme_fullbox\labels\train.cache... 8500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8500/8500 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 36.311.4 MB/s, size: 11.6 KB)


val: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme_fullbox\labels\val.cache... 1500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1500/1500 [00:00<?, ?it/s]


Plotting labels to runs\synthetic_extreme_yolov8n3\labels.jpg... 
optimizer: SGD(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs\synthetic_extreme_yolov8n3
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


244 epochs completed in 1.309 hours.
Optimizer stripped from runs\synthetic_extreme_yolov8n3\weights\last.pt, 6.2MB
Optimizer stripped from runs\synthetic_extreme_yolov8n3\weights\best.pt, 6.2MB

Validating runs\synthetic_extreme_yolov8n3\weights\best.pt...
Ultralytics 8.3.141  Python-3.9.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
Model summary (fused): 72 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:04<00:00,  5.17it/s]


                   all       1500       2986      0.984      0.956      0.977      0.961
                  blue        252        252      0.996       0.98      0.986      0.973
                   red        265        265          1      0.961      0.975      0.958
                orange        269        269      0.991      0.827      0.955      0.932
                yellow        242        242      0.862      0.952      0.948      0.937
                purple        261        261      0.991      0.981      0.994      0.977
                 brown        285        285          1      0.968      0.979      0.968
                  gray        277        277          1      0.972      0.986      0.963
                 black        283        283      0.997      0.982      0.993      0.984
                  pink        281        281      0.996      0.969       0.98      0.966
                 white        292        292      0.993      0.973      0.985      0.964
                 gree

val: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares_extreme_fullbox\labels\val.cache... 1500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1500/1500 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:04<00:00,  9.81it/s]


                   all       1500       2986      0.984      0.956      0.977      0.961
                  blue        252        252      0.996       0.98      0.986      0.973
                   red        265        265          1      0.961      0.975      0.958
                orange        269        269      0.991      0.827      0.955      0.934
                yellow        242        242      0.862      0.953      0.948      0.936
                purple        261        261      0.991      0.981      0.994      0.977
                 brown        285        285          1      0.968      0.979      0.968
                  gray        277        277          1      0.972      0.986      0.962
                 black        283        283      0.997      0.982      0.993      0.984
                  pink        281        281      0.996      0.969       0.98      0.966
                 white        292        292      0.993      0.973      0.985      0.964
                 gree

In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3070 Ti


In [2]:

pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


^C
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp39-cp39-win_amd64.whl (2449.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1


DEPRECATION: uiutil 1.38.0 has a non-standard dependency specifier future>=0.16.0attrs>=16.3.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of uiutil or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  You can safely remove it manually.


In [2]:
import os
from ultralytics import YOLO
import cv2

# Path to your trained YOLOv8 model (change to best.pt if needed)
model_path = r'runs\synthetic_extreme_yolov8n3\weights\best.pt'
img_folder = r'C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train'
output_folder = r'C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\train_bbox_vis'
os.makedirs(output_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO(model_path)

# Get class names from model
class_names = model.names

# Loop through all images
for img_name in os.listdir(img_folder):
    img_path = os.path.join(img_folder, img_name)
    # Only process image files
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif')):
        continue

    # Run inference
    results = model(img_path)
    result = results[0]  # first batch element

    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        label = f'{class_names[cls]} {conf:.2f}'

        # Draw rectangle
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
        # Put label
        cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    out_path = os.path.join(output_folder, img_name)
    cv2.imwrite(out_path, img)

print(f"Bounding box visualizations saved to: {output_folder}")



image 1/1 C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\blue_20250521-110915-682658.jpg: 224x224 2 whites, 12.4ms
Speed: 1.2ms preprocess, 12.4ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\blue_20250521-110915-962759.jpg: 224x224 1 brown, 1 white, 14.4ms
Speed: 0.9ms preprocess, 14.4ms inference, 3.9ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\blue_20250521-110920-333864.jpg: 224x224 1 white, 18.1ms
Speed: 0.7ms preprocess, 18.1ms inference, 5.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\blue_20250521-110925-724138.jpg: 224x224 3 browns, 26.0ms
Speed: 0.6ms preprocess, 26.0ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 C:\Users\fredr\BTH\Robotik Project\Rob

In [2]:
import cv2
import os
import glob

# CONFIGURATION
image_folder = r"C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train"
color_classes = [
    "black", "gray", "white", "red", "blue", "green", 
    "yellow", "orange", "brown", "purple", "pink", "cyan"
]
colors_bgr = [
    (0,0,0), (128,128,128), (255,255,255), (0,0,255), (255,0,0), (0,255,0),
    (0,255,255), (0,128,255), (19,69,139), (128,0,128), (203,192,255), (255,255,0)
]

# GLOBALS
drawing = False
ix, iy = -1, -1
annotations = []
current_class = 0
img = None
img_display = None

def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img, img_display, annotations, current_class

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        img_display = img.copy()
        draw_annotations()
        cv2.rectangle(img_display, (ix, iy), (x, y), colors_bgr[current_class], 2)

    elif event == cv2.EVENT_LBUTTONUP and drawing:
        drawing = False
        x1, x2 = sorted([ix, x])
        y1, y2 = sorted([iy, y])
        annotations.append((current_class, x1, y1, x2, y2))
        img_display = img.copy()
        draw_annotations()

def draw_annotations():
    global img_display
    for cls, x1, y1, x2, y2 in annotations:
        cv2.rectangle(img_display, (x1, y1), (x2, y2), colors_bgr[cls], 2)
        cv2.putText(img_display, color_classes[cls], (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors_bgr[cls], 2)
    menu_text = " | ".join([f"{i}-{name}" for i, name in enumerate(color_classes)])
    cv2.rectangle(img_display, (0,0), (img.shape[1], 60), (50,50,50), -1)
    cv2.putText(img_display, "Colors: " + menu_text, (5, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
    cv2.putText(img_display, f"Current Color: {color_classes[current_class]} (Press 'u' to undo, 's' to save, 'q' to quit)", 
                (5, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors_bgr[current_class], 1)

def save_annotations(img_path):
    global annotations, img
    h, w = img.shape[:2]
    label_path = os.path.splitext(img_path)[0] + ".txt"
    with open(label_path, "w") as f:
        for cls, x1, y1, x2, y2 in annotations:
            cx, cy = ((x1+x2)/2)/w, ((y1+y2)/2)/h
            bw, bh = (x2-x1)/w, (y2-y1)/h
            f.write(f"{cls} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")
    print(f"Annotations saved to: {label_path}")

def main():
    global current_class, annotations, img, img_display

    image_files = sorted(glob.glob(os.path.join(image_folder, "*.jpg")) +
                         glob.glob(os.path.join(image_folder, "*.png")))

    for img_path in image_files:
        annotations = []
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error loading {img_path}, skipping...")
            continue
        img_display = img.copy()

        cv2.namedWindow("Image Annotator", cv2.WINDOW_NORMAL)
        cv2.setMouseCallback("Image Annotator", draw_rectangle)
        draw_annotations()

        print(f"\nAnnotating image: {os.path.basename(img_path)}")

        while True:
            cv2.imshow("Image Annotator", img_display)
            key = cv2.waitKey(10) & 0xFF

            # Color selection
            if ord('0') <= key <= ord('9'):
                idx = key - ord('0')
                if idx < len(color_classes):
                    current_class = idx
                    img_display = img.copy()
                    draw_annotations()

            elif key == ord('a') and len(color_classes) > 10:
                current_class = 10
                img_display = img.copy()
                draw_annotations()

            elif key == ord('b') and len(color_classes) > 11:
                current_class = 11
                img_display = img.copy()
                draw_annotations()

            elif key == ord('u'):  # Undo last annotation
                if annotations:
                    annotations.pop()
                    img_display = img.copy()
                    draw_annotations()
                    print("Last annotation removed.")

            elif key == ord('s'):  # Save annotations
                save_annotations(img_path)
                break

            elif key == ord('q'):  # Quit tool
                cv2.destroyAllWindows()
                print("Exiting annotation tool.")
                return

        cv2.destroyAllWindows()

    print("All images annotated successfully.")

if __name__ == "__main__":
    main()



Annotating image: blue_20250521-110915-682658.jpg
Annotations saved to: C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\blue_20250521-110915-682658.txt

Annotating image: blue_20250521-110915-962759.jpg
Annotations saved to: C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\blue_20250521-110915-962759.txt

Annotating image: blue_20250521-110920-333864.jpg
Annotations saved to: C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\blue_20250521-110920-333864.txt

Annotating image: blue_20250521-110925-724138.jpg
Annotations saved to: C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\blue_20250521-110925-724138.txt

Annotating image: blue_20250521-110926-021988.jpg
Annotations saved to: C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train\blue_20250521-110926-021988.txt

Annotating image: blue_20250521-110926-300071.jpg
Annotations saved to: C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dat

In [7]:
import torch
import os
from glob import glob
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import shutil

# === CUDA CHECK ===
print("CUDA available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.current_device(), torch.cuda.get_device_name(torch.cuda.current_device()))

# === PATHS ===
dataset_root = r"C:/Users/fredr/BTH/Robotik Project/Robotics/yolo_dataset"
all_images_dir = os.path.join(dataset_root, "images", "train")  # original full set is here
all_labels_dir = os.path.join(dataset_root, "labels", "train")

split_dirs = {
    "images/train": os.path.join(dataset_root, "images", "split_train"),
    "images/val": os.path.join(dataset_root, "images", "split_val"),
    "labels/train": os.path.join(dataset_root, "labels", "split_train"),
    "labels/val": os.path.join(dataset_root, "labels", "split_val"),
}
for path in split_dirs.values():
    os.makedirs(path, exist_ok=True)

# === LOAD & SPLIT ===
images = glob(os.path.join(all_images_dir, "*.jpg"))
train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

def copy_split(images_list, split):
    img_dest = os.path.join(dataset_root, "images", f"split_{split}")
    lbl_dest = os.path.join(dataset_root, "labels", f"split_{split}")
    for img_path in images_list:
        base = os.path.basename(img_path)
        lbl_path = os.path.join(all_labels_dir, base.replace(".jpg", ".txt"))
        shutil.copy2(img_path, os.path.join(img_dest, base))
        if os.path.exists(lbl_path):
            shutil.copy2(lbl_path, os.path.join(lbl_dest, os.path.basename(lbl_path)))

copy_split(train_imgs, "train")
copy_split(val_imgs, "val")

# === CREATE YAML ===
yaml_path = os.path.join(dataset_root, "split_dataset.yaml")
dataset_root_fixed = dataset_root.replace('\\', '/')

with open(yaml_path, "w") as f:
    f.write(f"""\
path: {dataset_root_fixed}
train: images/split_train
val: images/split_val
names:
  0: blue
  1: red
  2: green
  3: yellow
  4: purple
  5: brown
  6: gray
  7: pink
  8: white
""")


# === TRAIN ===
model = YOLO("yolov8n.pt")
results = model.train(
    data=yaml_path,
    epochs=300,
    imgsz=224,
    patience=50,
    batch=32,
    lr0=0.001,
    optimizer="SGD",
    device=0,
    project="runs",
    name="synthetic_extreme_yolov8n_split",
    cos_lr=True
)

# === VALIDATE ===
metrics = model.val()


CUDA available? True
Device: 0 NVIDIA GeForce RTX 3070 Ti
New https://pypi.org/project/ultralytics/8.3.142 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.141  Python-3.9.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=C:/Users/fredr/BTH/Robotik Project/Robotics/yolo_dataset\split_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, mode

train: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\labels\split_train... 48 images, 0 backgrounds, 0 corrupt: 100%|██████████| 48/48 [00:00<00:00, 1777.97it/s]

train: New cache created: C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\labels\split_train.cache


val: Fast image access  (ping: 0.10.0 ms, read: 2.80.3 MB/s, size: 13.5 KB)


val: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\labels\split_val... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<00:00, 1333.39it/s]

...


178 epochs completed in 0.016 hours.
Optimizer stripped from runs\synthetic_extreme_yolov8n_split\weights\last.pt, 6.2MB
Optimizer stripped from runs\synthetic_extreme_yolov8n_split\weights\best.pt, 6.2MB

Validating runs\synthetic_extreme_yolov8n_split\weights\best.pt...
Ultralytics 8.3.141  Python-3.9.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
Model summary (fused): 72 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 19.04it/s]


                   all         12         12      0.975          1      0.995      0.706
                  blue          6          6      0.951          1      0.995      0.659
                yellow          6          6      0.998          1      0.995      0.753
Speed: 0.0ms preprocess, 0.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\synthetic_extreme_yolov8n_split
Ultralytics 8.3.141  Python-3.9.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
Model summary (fused): 72 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 202.546.5 MB/s, size: 14.0 KB)


val: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\labels\split_val.cache... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.99s/it]


                   all         12         12      0.975          1      0.995      0.706
                  blue          6          6      0.951          1      0.995      0.659
                yellow          6          6      0.998          1      0.995      0.753
Speed: 0.3ms preprocess, 4.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\synthetic_extreme_yolov8n_split2


In [8]:
import os
import cv2

# === Configurations ===
image_dir = r"C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train"
label_dir = r"C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\labels\train"
output_dir = r"C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\visualized"

# YOLO class names
class_names = [
    "blue", "red", "green", "yellow", "purple",
    "brown", "gray", "pink", "white"
]

# Assign colors for each class
class_colors = {
    name: (int(255 * (i / len(class_names))), 100, 255 - int(255 * (i / len(class_names))))
    for i, name in enumerate(class_names)
}

os.makedirs(output_dir, exist_ok=True)

# === Function to draw boxes ===
def draw_yolo_boxes(img_path, label_path, output_path):
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    if not os.path.exists(label_path):
        print(f"Warning: No label for {os.path.basename(img_path)}")
        return

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cls_id, x_center, y_center, box_w, box_h = map(float, parts)
            cls_id = int(cls_id)

            # Convert YOLO to pixel coordinates
            x1 = int((x_center - box_w / 2) * w)
            y1 = int((y_center - box_h / 2) * h)
            x2 = int((x_center + box_w / 2) * w)
            y2 = int((y_center + box_h / 2) * h)

            color = class_colors.get(class_names[cls_id], (0, 255, 0))
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, class_names[cls_id], (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imwrite(output_path, img)


# === Loop through all images ===
for filename in os.listdir(image_dir):
    if not filename.endswith(".jpg"):
        continue

    img_path = os.path.join(image_dir, filename)
    label_path = os.path.join(label_dir, filename.replace(".jpg", ".txt"))
    output_path = os.path.join(output_dir, filename)

    draw_yolo_boxes(img_path, label_path, output_path)

print("✅ Bounding boxes visualized and saved.")


✅ Bounding boxes visualized and saved.
